<a href="https://colab.research.google.com/github/yastiaisyah/DataSynthesis/blob/main/adversarial_autoencoder_realfake2_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from torchvision.datasets import ImageFolder
from torchvision import transforms
import numpy as np
import pandas as pd
from PIL import Image  # Import PIL untuk memproses gambar
from google.colab import drive

drive.mount('/content/drive')

# Define data transforms
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize gambar ke ukuran yang diinginkan (misalnya 64x64)
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalisasi nilai pixel
])

# Load dataset gambar wajah nyata dan palsu
sdir = '/content/drive/MyDrive/archive/real_and_fake_face/'

filepaths = []
labels = []
classlist = os.listdir(sdir)

# Iterasi melalui kelas (nyata dan palsu)
for klass in classlist:
    classpath = os.path.join(sdir, klass)

    # Pastikan ini adalah direktori (bukan file)
    if os.path.isdir(classpath):
        flist = os.listdir(classpath)

        # Iterasi melalui file dalam kelas ini
        for f in flist:
            fpath = os.path.join(classpath, f)
            filepaths.append(fpath)
            labels.append(klass)

# Buat Series dari filepaths dan labels
Fseries = pd.Series(filepaths, name='filepaths')
Lseries = pd.Series(labels, name='labels')

# Gabungkan Series menjadi DataFrame
df = pd.concat([Fseries, Lseries], axis=1)

# Cetak beberapa data pertama dan statistik kelas
print(df.head())
print(df['labels'].value_counts())

# Parameters
mb_size = 32
z_dim = 5
X_dim = 28 * 28
h_dim = 128
lr = 1e-3

# Encoder
Q = nn.Sequential(
    nn.Linear(64*64*3, h_dim),
    nn.ReLU(),
    nn.Linear(h_dim, z_dim)
)

# Decoder
P = nn.Sequential(
    nn.Linear(z_dim, h_dim),
    nn.ReLU(),
    nn.Linear(h_dim, 64*64*3),
    nn.Tanh()
)

# Discriminator
D = nn.Sequential(
    nn.Linear(z_dim, h_dim),
    nn.ReLU(),
    nn.Linear(h_dim, 1),
    nn.Sigmoid()
)

# Loss functions
criterion = nn.BCELoss()
z_sample = Variable(torch.randn(mb_size, z_dim))
D_optimizer = optim.Adam(D.parameters(), lr=lr)
G_optimizer = optim.Adam(list(Q.parameters()) + list(P.parameters()), lr=lr)

P_solver = optim.Adam(P.parameters(), lr=lr)  # Optimizer untuk model decoder (P)
Q_solver = optim.Adam(Q.parameters(), lr=lr)  # Optimizer untuk model encoder (Q)

def reset_grad():
    Q.zero_grad()
    P.zero_grad()
    D.zero_grad()

# Training
for it in range(100000):
    indices = np.random.randint(0, len(df), mb_size)

    # Mengubah ukuran gambar menjadi 28x28 dan normalisasi ke dalam kisaran [0, 1]
    X = torch.cat([transform(Image.open(df['filepaths'].iloc[i]).resize((28, 28)).convert('L')).view(1, -1) for i in indices])

    Y = torch.LongTensor([int(df['labels'].iloc[i] == 'training_real') for i in indices])  # Membaca label dari DataFrame

    # Reconstruction phase
    z_sample = Q(X)
    X_sample = P(z_sample)

    # Use BCELoss for binary cross entropy
    recon_loss = nn.BCELoss()(X_sample, X)

    recon_loss.backward()
    P_solver.step()
    Q_solver.step()
    reset_grad()

    # Print and plot every now and then
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {:.4}; G_loss: {:.4}; recon_loss: {:.4}'
              .format(it, D_loss.item(), G_loss.item(), recon_loss.item()))

        samples = P(z_real).data.numpy()[:16]

        fig = plt.figure(figsize=(4, 4))
        gs = gridspec.GridSpec(4, 4)
        gs.update(wspace=0.05, hspace=0.05)

        for i, sample in enumerate(samples):
            ax = plt.subplot(gs[i])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

        if not os.path.exists('out/'):
            os.makedirs('out/')

        plt.savefig('out/{}.png'
                    .format(str(cnt).zfill(3)), bbox_inches='tight')
        cnt += 1
        plt.close(fig)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
                                           filepaths         labels
0  /content/drive/MyDrive/archive/real_and_fake_f...  training_fake
1  /content/drive/MyDrive/archive/real_and_fake_f...  training_fake
2  /content/drive/MyDrive/archive/real_and_fake_f...  training_fake
3  /content/drive/MyDrive/archive/real_and_fake_f...  training_fake
4  /content/drive/MyDrive/archive/real_and_fake_f...  training_fake
training_real    1081
training_fake     975
Name: labels, dtype: int64


RuntimeError: ignored